In [1]:
import numpy as np
from load_mnist import MNIST

train_images, train_labels=MNIST(path="../Ehsan/Dataset/HW2/Q5",return_type="numpy",mode="vanilla").load_training()
test_images, test_labels=MNIST(path="../Ehsan/Dataset/HW2/Q5",return_type="numpy",mode="vanilla").load_testing()

for data in [test_images,train_images]:
    print("shape",data.shape)
    print("mean",data.mean())
    print("max",data.max())
    print("min",data.min())


shape (10000, 784)
mean 33.7912244898
max 255
min 0
shape (60000, 784)
mean 33.3184214498
max 255
min 0


In [2]:
def to_one_hat(y):
    o = np.zeros((len(y), 10))
    for i in range(len(y)):
        o[i][y[i] - 1] = 1
    return o

test_labels = to_one_hat(test_labels)
train_labels = to_one_hat(train_labels)

In [12]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import random
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from sklearn.model_selection import train_test_split
import time

## Define paramaters for the model

n_epochs = 5
batch_size = 60 
hidden_size = 100
learning_rate = tf.Variable(0.5)
std_dev = tf.Variable(10.0)
# regulation_rate = 1e-4

def fullLayer(input_data, output_size, act, std):    
    ##defining the full linear Layer here
    w = tf.Variable(tf.random_normal([input_data.get_shape().as_list()[1], output_size],mean = 0.0, stddev = std))
    b = tf.Variable(tf.random_normal([output_size], stddev = std))
    return act(tf.matmul(input_data, w) + b)

##define placeholder
## all image are 28 * 28 so x has 784 dimension
X = tf.placeholder(tf.float32, [batch_size, 784], name='X_placeholder')
Y = tf.placeholder(tf.float32, [batch_size, 10], name='Y_placeholder')




for std in [0.1, 1, 10, 20]:
    

    ## the layers
    layer1 = fullLayer(X, hidden_size, tf.nn.sigmoid, std)
    layer2 = fullLayer(layer1, hidden_size, tf.nn.sigmoid, std)
    layer3 = fullLayer(layer2, hidden_size, tf.nn.sigmoid, std)
    logits = fullLayer(layer3, 10, tf.nn.sigmoid, std)

    ## defining loss function
    ## use cross entropy of softmax of logits as the loss function
    entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name='loss')
    ## computes the mean over all the examples in the batch
    loss = tf.reduce_mean(entropy) 
    # + regulation_rate*tf.nn.l2_loss(w)  

    ##defining optimizer
    ## using gradient descent with learning rate of 0.5 to minimize loss
    gradient = tf.train.GradientDescentOptimizer(learning_rate)
    optimizer = gradient.minimize(loss)

    ##the prediction we made
    preds = tf.nn.softmax(logits)
    ##check how many of them are correct arg maxx is used because Y is one hat
    correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

    best_acc = 0
    best_hidden = 0

    ##just some config for not getting whole server
    gpu_options = tf.GPUOptions(allow_growth=True)
    config = tf.ConfigProto(gpu_options=gpu_options)
    sess = tf.Session(config = config)
    ## to visualize using TensorBoard
    #writer = tf.summary.FileWriter('./graphs/mnist/c', sess.graph)
    ##starting time
    start_time = time.time()
    ##initialize the variables
    sess.run(tf.global_variables_initializer())

    n_batches = int(60000 / batch_size)
    for i in range(n_epochs):  # train the model n_epochs times
        total_loss = 0
        total_acc = 0

        conc = list(zip(train_images, train_labels))
        random.shuffle(conc)
        train_images, train_labels = zip(*conc)

        for j in range(n_batches):
            X_batch, Y_batch = train_images[j* batch_size:(j+1)*batch_size], train_labels[j* batch_size:(j+1)*batch_size]
            ##training batches
            _, loss_batch, acc_batch = sess.run([optimizer, loss, accuracy], feed_dict={X: X_batch, Y: Y_batch})
            total_loss += loss_batch
            total_acc += acc_batch
            if j % 200 == 199:
                print('step {}, with std {}, Average loss : {}, Accuracy : {:.6f}'.format(i * n_batches + j, std, total_loss / 200, total_acc / batch_size / 200))
                
                
                total_loss = 0
                total_acc = 0

    print('Total time: {0} seconds'.format(time.time() - start_time))
    print('Optimization Finished!')

    # test the model
    ##number of test batches
    n_batches = int(10000 / batch_size)
    total_correct_preds = 0

    for i in range(n_batches):
        ##test batches
        X_batch, Y_batch = test_images[i* batch_size:(i+1)*batch_size], test_labels[i* batch_size:(i+1)*batch_size]
        accuracy_batch = sess.run([accuracy], feed_dict={X: X_batch, Y: Y_batch})
        total_correct_preds += accuracy_batch[0]

    print('with std {}, Accuracy {}'.format(std, total_correct_preds / 10000))
    
    sess.close()

step 199, with std 0.1, Average loss : 2.295365102291107, Accuracy : 0.133667
step 399, with std 0.1, Average loss : 2.183388706445694, Accuracy : 0.365417
step 599, with std 0.1, Average loss : 1.9598277503252028, Accuracy : 0.535333
step 799, with std 0.1, Average loss : 1.8422295576334, Accuracy : 0.593167
step 999, with std 0.1, Average loss : 1.7974943262338638, Accuracy : 0.583250
step 1199, with std 0.1, Average loss : 1.7824984139204025, Accuracy : 0.567000
step 1399, with std 0.1, Average loss : 1.7788734871149063, Accuracy : 0.574500
step 1599, with std 0.1, Average loss : 1.7663910472393036, Accuracy : 0.541083
step 1799, with std 0.1, Average loss : 1.7622174525260925, Accuracy : 0.532917
step 1999, with std 0.1, Average loss : 1.7550599414110184, Accuracy : 0.529333
step 2199, with std 0.1, Average loss : 1.7568813240528107, Accuracy : 0.523833
step 2399, with std 0.1, Average loss : 1.766355848312378, Accuracy : 0.508917
step 2599, with std 0.1, Average loss : 1.765592126